In [3]:
import numpy as np
from time import time
from QAP_Heuristic import QAP_Heuristic
from QAP_Tester import QAP_Hueristic_Tester

In [9]:
class TabuSearch(QAP_Heuristic):
    def __init__(self, w, d, tabu_size=100) -> None:
        super().__init__(w, d)
        self.tabu_size = tabu_size

    def __str__(self):
        return "TabuSearch"

    # auxiliary methods
    def generate_neighbors(self, solution):
        neighbors = []
        for i in range(len(solution)):
            for j in range(i+1, len(solution)):
                neighbor = solution.copy()
                neighbor[i], neighbor[j] = solution[j], solution[i]
                neighbors.append(neighbor)
        return neighbors

    # implements the solve method via the Tabu Search algorithm
    def solve(self, n_iter):
        
        best_solution = np.random.permutation(self.n).tolist()  # the .tolist() is janky, but it doesnt work with out it for some reason
        best_cost = self.cost(best_solution)
        
        current_solution = best_solution
        current_cost = best_cost
        
        tabu_list = []

        for _ in range(n_iter):

            if time() > self.MAX_CPU_TIME: break

            neighbors = self.generate_neighbors(current_solution)
            best_neighbor = None
            best_neighbor_cost = float('inf')
            
            for neighbor in neighbors:
                if neighbor not in tabu_list:
                    cost = self.cost(neighbor)
                    if cost < best_neighbor_cost:
                        best_neighbor_cost = cost
                        best_neighbor = neighbor
            
            if best_neighbor:
                tabu_list.append(best_neighbor)
                if len(tabu_list) > self.tabu_size:
                    tabu_list.pop(0)
                
                current_solution = best_neighbor
                current_cost = best_neighbor_cost
                
                if current_cost < best_cost:
                    best_cost = current_cost
                    best_solution = current_solution

        return best_solution

    # greedy constructive heuristic to formulate an initial solution
    def greedy_qap(self):
        F, D = self.W.copy(), self.D.copy()
        
        num_facilities = F.shape[0]
        remaining_facilities = set(range(num_facilities))
        remaining_locations = set(range(num_facilities))
        perm = [-1 for _ in range(num_facilities)]
        
        for i in range(num_facilities):
            best_cost = float('inf')
            best_facility, best_location = -1, -1
            
            for f in remaining_facilities:
                for l in remaining_locations:
                    current_cost = self.cost(perm)
                    
                    if current_cost < best_cost:
                        best_cost = current_cost
                        best_facility, best_location = f, l
                        
            perm[best_location] = best_facility
            remaining_facilities.remove(best_facility)
            remaining_locations.remove(best_location)
        
        return perm

### Automated Testing

Automatic Testing for the Hueristic on all instances of the problem

In [10]:
instance_path = "../QAPInstances/"
solution_path = "../QAPSolns/"
write_to_path = "../results/"

In [11]:
tester = QAP_Hueristic_Tester(heuristic=TabuSearch, 
                              instance_path=instance_path,
                              soln_path=solution_path,
                              write_to_path=write_to_path,
                              )

In [13]:
tester.test_hueristic(n_iters=10_000, n_trials=5, tai_only=True, tabu_size=100)